Bloco 1: Instalação de Dependências

In [ ]:
import sys
!{sys.executable} -m pip install pyarrow

Bloco 2: Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import json

Bloco 3: Carregamento dos Dados Processados

In [ ]:
caminho_dados_limpos = '../data/processed/dados_limpos.parquet'
df = pd.read_parquet(caminho_dados_limpos)

print("Dados limpos carregados com sucesso!")
display(df.head())

Bloco 4: Carregar e Aplicar Aliases (dos Mapas JSON)

In [ ]:
# Carregar os dicionários de mapeamento
with open('../data/processed/mapa_clientes_alias.json', 'r') as f:
    mapa_clientes = json.load(f)
with open('../data/processed/mapa_locais_alias.json', 'r') as f:
    mapa_locais = json.load(f)

print("Mapeamentos de alias carregados.")

# Aplicar os aliases para criar novas colunas legíveis
df['nome_cliente'] = df['fk_contact'].map(mapa_clientes)
df['origem_alias'] = df['place_origin_departure'].map(mapa_locais)
df['destino_alias'] = df['place_destination_departure'].map(mapa_locais)

# Criar a coluna 'produto' (o trecho legível)
df['produto'] = df['origem_alias'] + ' -> ' + df['destino_alias']

print("Colunas de alias criadas com sucesso!")
display(df[['nome_cliente', 'produto']].head())

Bloco 5: Engenharia de Features de Comportamento

In [ ]:
# Feature 1: tipo_viagem (Ida e Volta vs. Só Ida)
df['tipo_viagem'] = 'Ida e Volta'
df.loc[df['place_origin_return'] == '0', 'tipo_viagem'] = 'Só Ida'

# Feature 2: viaja_sozinho
df['viaja_sozinho'] = np.where(df['total_tickets_quantity_success'] == 1, 'Sim', 'Não')

# Feature 3: compra_fds (Fim de Semana)
df['compra_fds'] = np.where(df['datetime_purchase'].dt.dayofweek >= 5, 'Sim', 'Não')

# Feature 4: compra_alta_temporada
meses_alta_temporada = [12, 1, 7] # Exemplo: Dezembro, Janeiro e Julho
df['compra_alta_temporada'] = np.where(df['datetime_purchase'].dt.month.isin(meses_alta_temporada), 'Sim', 'Não')

print("Novas features de comportamento criadas. Amostra:")
display(df[['fk_contact', 'tipo_viagem', 'viaja_sozinho', 'compra_fds', 'compra_alta_temporada']].head())

Bloco 6: Análise RFM (Agregação por Cliente)

In [ ]:
# Usamos o dia seguinte à última compra registrada para garantir que todas as recências sejam positivas.
snapshot_date = df['datetime_purchase'].max() + pd.Timedelta(days=1)

# Agrupando por cliente (fk_contact) e calculando RFM
df_rfm = df.groupby('fk_contact').agg(
    Recencia=('datetime_purchase', lambda date: (snapshot_date - date.max()).days), # Recência
    Frequencia=('nk_ota_localizer_id', 'nunique'), # Frequência (compras únicas)
    ValorMonetario=('gmv_success', 'sum') # Valor Monetário
)

# Renomeando as colunas para o padrão RFM
df_rfm.rename(columns={'datetime_purchase': 'Recencia',
                       'nk_ota_localizer_id': 'Frequencia',
                       'gmv_success': 'ValorMonetario'}, inplace=True)

print("DataFrame RFM (agregado por cliente) criado com sucesso:")
display(df_rfm.head())

Bloco 7: Persistência dos Resultados (Saídas Finais)

In [ ]:
# --- Saída A: O log de VENDAS (transacional) completo, agora limpo e com aliases.
# (Usado pelo app.py e pelo Notebook 06_Recommendation_Model)
print("Alinhando e salvando o arquivo de vendas para o frontend...")
caminho_sales = '../data/redis/sales.parquet'

# Renomeando colunas-chave para o padrão do app
df_para_app = df.rename(columns={
    'fk_contact': 'id_cliente',
    'datetime_purchase': 'data_venda',
    'gmv_success': 'valor_venda'
})

df_para_app.to_parquet(caminho_sales, index=False)
print(f"--> Arquivo 'sales.parquet' salvo em: '{caminho_sales}'")


# --- Saída B: O arquivo AGREGADO (RFM)
# (Usado pelo Notebook 04_Segmentation_Model)
print("\nSalvando o arquivo com as features RFM...")
caminho_rfm = '../data/curated/rfm_features.parquet'
df_rfm.to_parquet(caminho_rfm)
print(f"--> Arquivo RFM ('rfm_features.parquet') salvo em: '{caminho_rfm}'")